In [1]:
!pip install azure-cosmos requests

In [2]:
import requests
import os
import json
import azure.cosmos.documents as documents
import azure.cosmos.cosmos_client as cosmos_client
import azure.cosmos.exceptions as exceptions
from azure.cosmos.partition_key import PartitionKey
from datetime import datetime
from random import randint
import time  # Importa el módulo necesario para el delay


In [3]:
import requests
import json

def fetch_vulnerability_data(keyword):
    """
    Fetches vulnerability data for a given keyword from the NVD database and returns it as a JSON object.
    
    Args:
    keyword (str): The keyword to search for vulnerabilities related to it.
    
    Returns:
    dict: A dictionary containing the vulnerability data.
    """
    
    url = f"https://services.nvd.nist.gov/rest/json/cves/2.0?keywordSearch={keyword}&keywordExactMatch"

    payload = {}
    headers = {}

    # Make the request
    response = requests.request("GET", url, headers=headers, data=payload)

    # Convert the text response to a JSON object
    response_json = response.json()

    # Specifically extract the 'vulnerabilities' portion of the response
    vulnerabilities = response_json.get('vulnerabilities', [])

    # Prepare a dictionary to hold all vulnerability data
    vulnerability_dict = {}

    # Iterate through each vulnerability to extract and structure specific information
    for vulnerability in vulnerabilities:
        cve = vulnerability.get('cve', {})
        cve_id = cve.get('id', 'Unknown CVE')  # Use 'Unknown CVE' if no ID is provided
        cve_data = {
            "Published Date": cve.get('published'),
            "Last Modified Date": cve.get('lastModified'),
            "Descriptions": [{"Language": desc.get('lang'), "Description": desc.get('value')} for desc in cve.get('descriptions', [])]
        }

        # Access the CVSS metrics if available
        metrics = cve.get('metrics', {})
        cvssMetricV2 = metrics.get('cvssMetricV2', [])
        cvssMetricV31 = metrics.get('cvssMetricV31', [])

        # Try to find CVSS v2 data first
        if cvssMetricV2:
            cve_data["Base Severity"] = cvssMetricV2[0].get('baseSeverity', 'No severity provided')
            cve_data["Impact Score"] = cvssMetricV2[0].get('impactScore', 'No impact score provided')
        # If not available, look for CVSS v3.1 data
        elif cvssMetricV31:
            cvssData = cvssMetricV31[0].get('cvssData', {})
            cve_data["Base Severity"] = cvssData.get('baseSeverity', 'No severity provided')
            cve_data["Impact Score"] = cvssMetricV31[0].get('impactScore', 'No impact score provided')

        # Add this CVE's data to the main dictionary, using the CVE ID as the key
        vulnerability_dict[cve_id] = cve_data

    return vulnerability_dict

In [4]:
def secDataFormat(keyword, vulnerability_data):
    """
    Formats the vulnerability data with additional keys and returns it as JSON.

    Args:
    keyword (str): The keyword used to search for vulnerabilities.
    vulnerability_data (dict): The vulnerability data fetched from NVD.

    Returns:
    dict: A dictionary containing the formatted vulnerability data.
    """
    partition_key = 1 if vulnerability_data else 0

    formatted_data = {
        "id": keyword,
        "partitionKey": partition_key,
        "vulnerabilities": vulnerability_data,
    }
    return formatted_data

In [8]:
# Example usage:
#keyword = ["Microsoft Outlook", "Adobe Acrobat
keyword = ['MySQL Installer - Community',
'Microsoft Visual C++ 2013 Redistributable (x86) - 12.0.40664',
'AMD Ryzen Balanced Driver',
'vs_filehandler_x86',
'Google Update Helper',
'AMD SBxxx SMBus Driver',
'vs_BlendMsi',
'Promontory_GPIO Driver',
'Integration Services',
'vs_FileTracker_Singleton',
'Microsoft .NET Framework 4.8 Targeting Pack',
'vs_minshellmsires',
'Microsoft ASP.NET Diagnostic Pack for Visual Studio',
'icecap_collectionresourcesx64',
'Microsoft Visual C++ 2022 X86 Additional Runtime - 14.36.32532',
'AMD Settings',
'IntelliTraceProfilerProxy',
'Intel Driver && Support Assistant',
'Microsoft ASP.NET Core 7.0.9 Targeting Pack (x86)',
'Google SketchUp 8',
'Microsoft Garage Mouse without Borders',
'Microsoft Visual C++ 2013 x86 Additional Runtime - 12.0.40664',
'Microsoft .NET Runtime - 7.0.9 (x86)',
'vs_tipsmsi',
'vs_devenvsharedmsi',
'Paquete de compatibilidad de Microsoft .NET Framework 4.7.2 (español)',
'Microsoft Visual Studio Setup WMI Provider',
'AMD GPIO2 Driver',
'Entity Framework 6.2.0 Tools  for Visual Studio 2022',
'Microsoft .NET Host - 7.0.9 (x86)',
'Microsoft Visual C++ 2013 Redistributable (x86) - 12.0.30501',
'AMD_Chipset_Drivers',
'Microsoft Windows Desktop Runtime - 6.0.20 (x86)',
'Browser for SQL Server 2022',
'Discord',
'Postman x86_64 10.24.16',
'Riot Client ',
'VALORANT',
'Microsoft Teams classic',
'Microsoft Visual Studio Code (User)',
'Python 3.12.2 (64-bit)',
'AMPL 0.0.1',
'Discord',
'Postman x86_64 10.24.16',
'Riot Client ',
'VALORANT',
'Microsoft Teams classic',
'Microsoft Visual Studio Code (User)',
'Python 3.12.2 (64-bit)',
'AMPL 0.0.1']


In [5]:
# Define un diccionario llamado "settings" que contiene la configuración para Azure Cosmos DB
settings = {
    'host': os.environ.get('ACCOUNT_HOST', ''),  # Obtiene el valor del host de la variable de entorno ACCOUNT_HOST, si no está definida, utiliza un valor predeterminado
    'master_key': os.environ.get('ACCOUNT_KEY', ''),  # Obtiene el valor de la clave maestra de la variable de entorno ACCOUNT_KEY, si no está definida, utiliza un valor predeterminado
    'database_id': os.environ.get('COSMOS_DATABASE', ''),  # Obtiene el nombre de la base de datos de la variable de entorno COSMOS_DATABASE, si no está definida, utiliza un valor predeterminado
    'container_id': os.environ.get('COSMOS_CONTAINER','')  # Obtiene el nombre del contenedor de la variable de entorno COSMOS_CONTAINER, si no está definida, utiliza un valor predeterminado
}

# Asigna los valores de la configuración a variables individuales para facilitar su uso
api_key = ""
HOST = settings['host']  # La URL del servicio de Azure Cosmos DB
MASTER_KEY = settings['master_key']  # La clave maestra para acceder al servicio de Azure Cosmos DB
DATABASE_ID = settings['database_id']  # El ID de la base de datos en Azure Cosmos DB
CONTAINER_ID = settings['container_id']  # El ID del contenedor en Azure Cosmos DB


In [6]:
def read_item(container, doc_id, account_number):
    """
    Esta función lee un ítem de la base de datos de Azure Cosmos DB utilizando el ID del documento y el número de cuenta como clave de partición.

    Parámetros:
    - container: El cliente del contenedor de Azure Cosmos DB.
    - doc_id: El ID del documento que se desea leer.
    - account_number: El número de cuenta que se utilizará como clave de partición.

    No retorna nada, pero imprime la información del ítem leído.
    """
    print('\nReading Item by Id\n')
    # Hace una búsqueda eficiente del ítem utilizando la clave de partición y el ID
    response = container.read_item(item=doc_id, partition_key=account_number)
    print('Item read by Id {0}'.format(doc_id))
    print('Partition Key: {0}'.format(response.get('partitionKey')))
    print('Description: {0}'.format(response.get('VulnerabilityDesc')))

# Creación del cliente para Azure Cosmos DB
client = cosmos_client.CosmosClient(HOST, {'masterKey': MASTER_KEY}, user_agent="CosmosDBPythonQuickstart", user_agent_overwrite=True)

# Intenta crear la base de datos, si ya existe, obtiene la instancia de la misma
try:
    db = client.create_database(id=DATABASE_ID)
    print(f"Database with id '{DATABASE_ID}' created.")
except exceptions.CosmosResourceExistsError:
    db = client.get_database_client(DATABASE_ID)
    print(f"Database with id '{DATABASE_ID}' already exists.")

# Intenta crear el contenedor, si ya existe, obtiene la instancia del mismo
try:
    container = db.create_container(id=CONTAINER_ID, partition_key=PartitionKey(path='/partitionKey'))
    print(f"Container with id '{CONTAINER_ID}' created.")
except exceptions.CosmosResourceExistsError:
    container = db.get_container_client(CONTAINER_ID)
    print(f"Container with id '{CONTAINER_ID}' already exists.")

Database with id 'Software' already exists.
Container with id 'Registros_cvss' already exists.


In [9]:
def main(keyword):
    for software_name in keyword:
        try:
            resultado_dict = secDataFormat(software_name, fetch_vulnerability_data(software_name))  # Utiliza el nombre del software como palabra clave
            if resultado_dict:  # Verifica si se obtuvieron resultados
                print(resultado_dict)
                response = container.upsert_item(body=resultado_dict)
                print("Ítem insertado o actualizado exitosamente.")
            else:
                print(f"No se encontraron vulnerabilidades para el software '{software_name}'")
        except Exception as e:
            print(f"Ocurrió un error al intentar insertar o actualizar el ítem para el software '{software_name}': {e}")
        
        time.sleep(10)  # Espera 10 segundos antes de continuar con el siguiente software

# Ejemplo de uso con una lista de nombres de software
main(keyword)


{'id': 'MySQL Installer - Community', 'partitionKey': 0, 'vulnerabilities': {}}
Ítem insertado o actualizado exitosamente.
{'id': 'Microsoft Visual C++ 2013 Redistributable (x86) - 12.0.40664', 'partitionKey': 0, 'vulnerabilities': {}}
Ítem insertado o actualizado exitosamente.
{'id': 'AMD Ryzen Balanced Driver', 'partitionKey': 0, 'vulnerabilities': {}}
Ítem insertado o actualizado exitosamente.
{'id': 'vs_filehandler_x86', 'partitionKey': 0, 'vulnerabilities': {}}
Ítem insertado o actualizado exitosamente.
{'id': 'Google Update Helper', 'partitionKey': 0, 'vulnerabilities': {}}
Ítem insertado o actualizado exitosamente.
{'id': 'AMD SBxxx SMBus Driver', 'partitionKey': 0, 'vulnerabilities': {}}
Ítem insertado o actualizado exitosamente.
{'id': 'vs_BlendMsi', 'partitionKey': 0, 'vulnerabilities': {}}
Ítem insertado o actualizado exitosamente.
{'id': 'Promontory_GPIO Driver', 'partitionKey': 0, 'vulnerabilities': {}}
Ítem insertado o actualizado exitosamente.
{'id': 'Integration Service